In [1]:
import numpy as np
import serial
import serial.tools.list_ports
from time import sleep
import sounddevice as sd

In [2]:
ports = serial.tools.list_ports.comports()
for i in range(len(ports)):
    print(ports[i])

/dev/cu.Bluetooth-Incoming-Port - n/a
/dev/cu.iworld-COM3 - n/a
/dev/cu.iPhonedeLeandro-Wireles - n/a


In [3]:
# Essa porta você deve escolher de acordo com sua Porta conectada
ser = serial.Serial(ports[3].device, baudrate=115200, timeout=1)

IndexError: list index out of range

In [4]:
def gerador_de_onda(frequencia, pontos_por_seg= 44100, duracao=0.7):
    '''
    Função que pega a frequência e duração do tempo de uma onda 
    como base de entrada e retorna um Array de valores com todos os 
    pontos no tempo.
    '''
    amplitude = 4096
    t = np.linspace(0, duracao, int(pontos_por_seg * duracao))
    som = amplitude * np.sin(2 * np.pi * frequencia * t)
    
    return som

In [5]:
def gerar_notas_piano(base_freq):
    '''
    base_freq = 261.63 é a frequência da nota C4 (Dó da quarta oitava)
    A Função retorna um dicionário para todas as frequências das notas 
    do Piano
    '''
    # Teclas brancas estão em Maiúsculo, teclas pretas (sustenido) estão em Minúsculo
    oitava = ['C', 'c', 'D', 'd', 'E', 'F', 'f', 'G', 'g', 'A', 'a', 'B'] 
    
    # Gerar dicionário com as frequências das oitavas, usando a base escolhida (C4)
    freq_notas = {oitava[i]: base_freq * pow(2,(i/12)) for i in range(len(oitava))}        
    freq_notas[''] = 0.0 # silêncio
    
    return freq_notas

In [6]:
def gerar_musica(notas_musicais, pontos_por_seg=44100, duracao=0.25, base_freq=261.63):
    '''
    Função que concatena todas as ondas (notas) 
    '''    
    freq_notas = gerar_notas_piano(base_freq)
    
    som = [gerador_de_onda(freq_notas[nota], pontos_por_seg, duracao) for nota in notas_musicais.split(',')]
    som = np.concatenate(som)
   # som = np.round(som * 16300 / np.max(som),2)
        
    return som

In [7]:
# Parâmetros
samplerate=40000
freq_notas = gerar_notas_piano(261.63)
acordes = list(freq_notas)
frequencias = list(freq_notas.values())
freq_min=250 
freq_max=505
tom = 'C'

In [8]:
freq_notas

{'C': 261.63,
 'c': 277.18732937722245,
 'D': 293.66974569918125,
 'd': 311.1322574981619,
 'E': 329.63314428399565,
 'F': 349.2341510465061,
 'f': 370.00069432367286,
 'G': 392.0020805232462,
 'g': 415.31173722644,
 'A': 440.00745824565865,
 'a': 466.1716632541139,
 'B': 493.8916728538229,
 '': 0.0}

In [22]:
while True:
    
    # Lendo as frequências convertidas que vem do microcontrolador
    dados_freq = int(ser.readline().decode('utf-8','ignore'))
    
    for nota in range(len(freq_notas)-1):
        if dados_freq >= frequencias[nota] and dados_freq <= frequencias[nota+1]:
            print('Acorde', acordes[nota], 'freq', round(frequencias[nota],2))
            tom = acordes[nota]
        # sí (B) está de fora, tem que colocar
        if dados_freq > freq_notas['B'] and dados_freq < freq_max:
            tom = 'B'
            print('Acorde B, freq', round(freq_notas['B'],2))

        if dados_freq <= freq_min or dados_freq >= freq_max:
            tom = ',,'

    musica = gerar_musica(notas_musicais=tom, 
                          pontos_por_seg=samplerate, duracao=0.7)
    sd.play(musica, samplerate)

Acorde C freq 261.63
Acorde C freq 261.63
Acorde C freq 261.63
Acorde c freq 277.19
Acorde D freq 293.67
Acorde d freq 311.13
Acorde E freq 329.63
Acorde F freq 349.23
Acorde f freq 370.0
Acorde G freq 392.0
Acorde g freq 415.31
Acorde A freq 440.01
Acorde a freq 466.17
Acorde a freq 466.17
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde B, freq 493.89
Acorde E freq 329.63
Acorde E freq 329.63
Acorde E freq 329.63
Acorde g freq 415.31
Acorde a freq 466.17
Acorde A freq 440.01
Acorde a freq 466.17
Acorde d freq 311.13
Acorde d fre

SerialException: read failed: [Errno 6] Device not configured